In [1]:
!pip install streamlit tensorflow pillow numpy joblib pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.9 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
!cp "/content/drive/MyDrive/pcos_models/efficientnetb4_pcos_best.h5" .
!cp "/content/drive/MyDrive/pcos_models/pcos_mlp_model.h5" .


In [5]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np
import joblib

# ---- Load models ----
@st.cache_resource
def load_models():
    image_model = load_model("efficientnetb4_pcos_best.h5")
    clinical_model = load_model("pcos_mlp_model.h5")
    scaler = None
    try:
        scaler = joblib.load("scaler.joblib")
    except:
        pass
    return image_model, clinical_model, scaler

image_model, clinical_model, scaler = load_models()

# ---- Preprocessing ----
def preprocess_image(img):
    img = img.convert("RGB").resize((224,224))
    arr = np.array(img)/255.0
    return np.expand_dims(arr, axis=0)

def preprocess_clinical(values):
    X = np.array([values], dtype=float)
    if scaler is not None:
        X = scaler.transform(X)
    return X

# ---- Prediction ----
def predict_image(img):
    x = preprocess_image(img)
    base_pred = image_model.predict(x)[0][0]
    # Apply your threshold
    label = "No PCOS" if base_pred > 0.5 else "PCOS"
    return label

def predict_clinical(values):
    X = preprocess_clinical(values)
    prob = clinical_model.predict(X)[0][0]
    # MLP model threshold = 0.5
    label = "PCOS" if prob > 0.5 else "No PCOS"
    return label

# ---- Streamlit UI ----
st.title(" Hybrid PCOS Detection System")

option = st.radio("Choose input type:", ["Image", "Clinical Data"])

if option=="Image":
    file = st.file_uploader("Upload ultrasound image", type=["jpg","jpeg","png"])
    if file:
        img = Image.open(file)
        st.image(img, caption="Uploaded Image", use_container_width=True)
        if st.button("Predict"):
            result = predict_image(img)
            st.success(f"Prediction: {result}")

elif option == "Clinical Data":
    st.write("Enter all clinical values:")

    # --- Basic details ---
    slno = st.number_input("Sl. No:")
    age = st.number_input("Age (yrs):")
    weight = st.number_input("Weight (Kg):")
    height = st.number_input("Height (Cm):")
    bmi = st.number_input("BMI:")

    blood_group = st.selectbox("Blood Group:", ["A", "B", "AB", "O"])
    pulse = st.number_input("Pulse rate (bpm):")
    rr = st.number_input("RR (breaths/min):")
    hb = st.number_input("Hb (g/dl):")

    cycle = st.selectbox("Cycle (R/I):", ["R", "I"])
    cycle_len = st.number_input("Cycle length (days):")
    marriage_yrs = st.number_input("Marriage Status (Yrs):")

    pregnant = st.selectbox("Pregnant (Y/N):", ["N", "Y"])
    abortions = st.number_input("No. of abortions:")

    # --- Hormones ---
    beta_hcg1 = st.number_input("I β-HCG (mIU/mL):")
    beta_hcg2 = st.number_input("II β-HCG (mIU/mL):")
    fsh = st.number_input("FSH (mIU/mL):")
    lh = st.number_input("LH (mIU/mL):")
    fsh_lh = st.number_input("FSH/LH:")
    hip = st.number_input("Hip (inch):")
    waist = st.number_input("Waist (inch):")
    whr = st.number_input("Waist:Hip Ratio:")
    tsh = st.number_input("TSH (mIU/L):")
    amh = st.number_input("AMH (ng/mL):")
    prl = st.number_input("PRL (ng/mL):")
    vitd = st.number_input("Vit D3 (ng/mL):")
    prg = st.number_input("PRG (ng/mL):")
    rbs = st.number_input("RBS (mg/dl):")

    # --- Symptoms (Y/N) ---
    weight_gain = st.selectbox("Weight gain (Y/N):", ["N", "Y"])
    hair_growth = st.selectbox("Hair growth (Y/N):", ["N", "Y"])
    skin_dark = st.selectbox("Skin darkening (Y/N):", ["N", "Y"])
    hair_loss = st.selectbox("Hair loss (Y/N):", ["N", "Y"])
    pimples = st.selectbox("Pimples (Y/N):", ["N", "Y"])
    fast_food = st.selectbox("Fast food (Y/N):", ["N", "Y"])
    reg_exercise = st.selectbox("Regular Exercise (Y/N):", ["N", "Y"])

    # --- Blood pressure & ultrasound features ---
    bp_sys = st.number_input("BP Systolic (mmHg):")
    bp_dia = st.number_input("BP Diastolic (mmHg):")
    follicle_l = st.number_input("Follicle No. (L):")
    follicle_r = st.number_input("Follicle No. (R):")
    avg_fsize_l = st.number_input("Avg. F size (L) (mm):")
    avg_fsize_r = st.number_input("Avg. F size (R) (mm):")
    endometrium = st.number_input("Endometrium (mm):")
    unnamed_44 = st.number_input("Unnamed: 44:")

    # --- Encode categorical values numerically ---
    yn_map = lambda x: 1 if x == "Y" else 0
    cycle_map = {"R": 1, "I": 0}
    blood_map = {"A": 1, "B": 2, "AB": 3, "O": 4}

    # --- Prepare input in correct order (43 features) ---
    input_data = [
        slno, age, weight, height, bmi,
        blood_map[blood_group],
        pulse, rr, hb,
        cycle_map[cycle],
        cycle_len, marriage_yrs,
        yn_map(pregnant), abortions,
        beta_hcg1, beta_hcg2, fsh, lh, fsh_lh,
        hip, waist, whr, tsh, amh, prl,
        vitd, prg, rbs,
        yn_map(weight_gain), yn_map(hair_growth), yn_map(skin_dark),
        yn_map(hair_loss), yn_map(pimples), yn_map(fast_food), yn_map(reg_exercise),
        bp_sys, bp_dia, follicle_l, follicle_r,
        avg_fsize_l, avg_fsize_r, endometrium, unnamed_44
    ]

    if st.button("Predict"):
        # Safety check
        st.write(f"Feature count: {len(input_data)}")
        label = predict_clinical(input_data)
        st.success(f"Prediction: {label}")


Writing app.py


In [6]:
!ngrok config add-authtoken 35I0poqLlmcvrtrIMuwesrBli5v_6pykjRpa6vZGqMVjZqNbn


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
# start the Streamlit server
!streamlit run app.py &>/content/logs.txt &


In [8]:
# create the public tunnel
from pyngrok import ngrok
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://karyn-consecrative-pamala.ngrok-free.dev" -> "http://localhost:8501">

In [ ]:
from pyngrok import ngrok
ngrok.kill()  # closes all active tunnels
